In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
test_passenger_ids = test_df['PassengerId']

In [ ]:
train_df.head()

In [ ]:
print(train_df.describe())

In [ ]:
print(train_df.isnull().sum())

Funzione di Analisi : gestisce i valori mancanti di Age,Fare con la media dei valori, Embraked è il porto di Imbarco e usiamo la moda ovvero il porto più frequente nel data set.
Creiamo una nuova colonna nel df che  riassume le informazioni  di SibSp(fratelli/cugini) e Parch(genitori/figli), sex viene convertito in 0 e 1 in base al sesso, pd.get_dummies appica il one_hot encoding a Embarked, e trasfroma la collonna in colonne diverse con valori che hanno 0 e 1 , dopo vengono troppate tutte le colonne che non servono al modello per fare le previsioni e infine ritorna il dataframe pulito.

In [ ]:
def preprocess_data(df):
    df['Age'].fillna(df['Age'].median(), inplace=True)
    df['Fare'].fillna(df['Fare'].median(), inplace=True)
    df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
    df['FamilySize'] = df['SibSp'] + df['Parch']
    df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
    df = pd.get_dummies(df, columns=['Embarked'], drop_first=True)
    df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp', 'Parch'], axis=1)
    return df

In [ ]:
train_processed = preprocess_data(train_df.copy())

In [ ]:
X = train_processed.drop('Survived', axis=1)
y = train_processed['Survived']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Dimensione Training Set: {X_train.shape[0]} campioni")
print(f"Dimensione Validation Set: {X_val.shape[0]} campioni")

# Primo Addestramento del modello 

In [ ]:
model = LogisticRegression(max_iter=2000, random_state=42)
model.fit(X_train, y_train)

In [ ]:
predictions_val = model.predict(X_val)
accuracy = accuracy_score(y_val, predictions_val)
print(f"\nAccuratezza sul set di validazione: {accuracy:.4f}")

In [ ]:
final_model = LogisticRegression(max_iter=2000, random_state=42)
final_model.fit(X, y)

In [ ]:
test_processed = preprocess_data(test_df.copy())
# Allinea le colonne (assicurati che il test set abbia le stesse colonne del training)
test_final = test_processed.reindex(columns=X.columns, fill_value=0)

In [ ]:
final_predictions = final_model.predict(test_final)
survived_df = pd.DataFrame({
    'PassengerId': test_passenger_ids,
    'Survived': final_predictions
})
survived_df.to_csv('survived.csv', index=False)

